In [123]:

from pathlib import Path
import pandas as pd
import os

In [124]:
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data
path = data_path / 'stock price data'
ticker_all = pd.read_csv(path/"finer_data_complete_stock.csv") 
stock = pd.read_csv(path/"company stock price daily"/'AAPL.csv') 
stock_bench = pd.read_csv(path/"benchmark price daily"/'^GSPC.csv')
ticker_all

,Symbol,Sector,Industry
0,A,Healthcare,Diagnostics & Research
1,AAL,Industrials,Airlines
2,AAN,Industrials,Rental & Leasing Services
3,AAON,Industrials,Building Products & Equipment
4,AAP,Consumer Cyclical,Specialty Retail
...,...,...,...
2112,ZEUS,Basic Materials,Steel
2113,ZION,Financial Services,Banks—Regional
2114,ZIONP,Financial Services,Banks—Regional
2115,ZUMZ,Consumer Cyclical,Apparel Retail


In [125]:
def volatility(stock, stock_bench, method="std", period=None):
    # Ensure the 'date' column is in datetime format
    df = stock.copy()
    df['date'] = pd.to_datetime(df['date'])
    # Set 'date' as the index
    df.set_index('date', inplace=True)

    df_bench = stock_bench.copy()
    df_bench['date'] = pd.to_datetime(df_bench['date'])

    # Use the entire dataset length as the period if not specified
    if period is None:
        period = len(df)
    
    if method == "std":
        # Calculate the rolling standard deviation, using a minimum of 1 to avoid division by zero
        rolling_stats = df['changePercent'].rolling(window=period).std()
    if method == "range":
        rolling_stats = (df['close'].rolling(window=period, min_periods=1).max() - df['close'].rolling(window=period).min())/df['close'].rolling(window=period).mean()
    if method == "beta":
        # Calculate covariance between stock and market returns
        covariance = pd.concat([df["changePercent"].reset_index(drop=True).rename("Stock return"), df_bench["changePercent"].rename("Market return")], axis=1).rolling(window=30).cov(pairwise=True).unstack()["Stock return", "Market return"]
        # Calculate variance of market returns
        market_variance = df_bench['changePercent'].rolling(window=period).var()
        rolling_stats = covariance / market_variance
    return rolling_stats.mean()

In [126]:
time_start = "2014-01-01"
time_end = "2016-01-01"
# ['Healthcare', 'Industrials', 'Consumer Cyclical', 'Technology','Financial Services', 'Real Estate', 'Basic Materials','Consumer Defensive', 'Energy', 'Utilities','Communication Services']
Sectors = ['Industrials', 'Consumer Cyclical', 'Technology','Financial Services', 'Real Estate', 'Basic Materials', 'Consumer Defensive', 'Communication Services']
method = "std"
period = 30
ticker_sector = ticker_all[ticker_all["Sector"].isin(Sectors)]
ticker_dict = {}
top = 10
for ticker in ticker_sector["Symbol"].unique():
    ticker_file = ticker+".csv"
    stock = pd.read_csv(path/"company stock price daily"/ticker_file)
    stock_time = stock[(stock["date"]>=time_start)&(stock["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)
    stock_bench_time = stock_bench[(stock_bench["date"]>=time_start)&(stock_bench["date"]<=time_end)].sort_values(by="date").reset_index(drop=True)
    ticker_dict[ticker] = volatility(stock_time, stock_bench_time, method="std", period=30)
top_ticker = [item[0] for item in sorted(ticker_dict.items(), key=lambda x: x[1])[:top]]

In [122]:
top_ticker

['DDT', 'BKT', 'ELA', 'VKQ', 'BTA', 'CHT', 'EVV', 'LEO', 'HSBC', 'DSM']